# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [2]:
!pip3 install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [3]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = "sk-f9d39c5a763a48cb9dff8d197a7b8aee"

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [18]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [4]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [5]:
len(text_lines)

72

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [5]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [6]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


生成一个测试嵌入并打印其维度和前几个元素。

In [7]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836056  0.07163018 -0.01130064 -0.03789344 -0.03320646 -0.01318444
 -0.03041711 -0.02269505 -0.02317867 -0.00426023]


In [8]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.0275297   0.06088526  0.00388529 -0.00215193 -0.02774976 -0.01186187
 -0.04020914 -0.06023425 -0.03813157  0.01002724]


## 将数据加载到 Milvus

### 创建 Collection

In [9]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo1.db")

collection_name = "my_rag_collection_mfd"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [10]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [11]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [12]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 181571.60it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [13]:
question = "在不动产交易中，如果未办理不动产登记，但买卖合同已经履行，买受人能否依据《民法典》主张物权的取得？"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [14]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [16]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###",
        0.6404271721839905
    ],
    [
        "二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [18]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [19]:
context

'第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以

In [20]:
question

'在不动产交易中，如果未办理不动产登记，但买卖合同已经履行，买受人能否依据《民法典》主张物权的取得？'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [22]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [21]:
USER_PROMPT

"\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus flush data?\n\nMilvus 

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [23]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据《民法典》第三百八十三条关于居住权的规定，"设立居住权，当事人应当依照法律规定办理登记。未经登记，不发生物权效力。" 这一原则同样适用于其他不动产物权的设立和转让。因此，在不动产交易中，即使买卖合同已经履行，如果未办理不动产登记，买受人不能依据《民法典》主张物权的取得，因为未经登记不发生物权效力。

<translated>
根据《民法典》第三百八十三条关于居住权的规定，"设立居住权，当事人应当依照法律规定办理登记。未经登记，不发生物权效力。" 这一原则同样适用于其他不动产物权的设立和转让。因此，在不动产交易中，即使买卖合同已经履行，如果未办理不动产登记，买受人不能依据《民法典》主张物权的取得，因为未经登记不发生物权效力。
</translated>
